__собираем предложения по квартирам__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/flat/avito' 
cian_raw_data_path = 'data/raw/flat/cian'

result_file_path = 'data/data_flat.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/flat/avito/avito_2022-08-26_17-29_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-29_13-16_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-31_17-42_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-01_14-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-02_13-04_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-05_14-02_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-06_13-09_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-07_12-39_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-08_13-36_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-12_13-44_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-13_13-55_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-15_10-27_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-16_15-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-19_12-36_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-22_09-48_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-26_11-20_raw.xlsx',
 'data/raw/flat/avito/avito_2022-10-08_20-34_raw.xlsx',
 'data/raw/flat/avito/avito_2022-10-14_15-48_raw

In [7]:
from lib.avito import AvitoDataCleanerRealtyFlat

data_avito = pd.concat([
        pd.read_excel(f)
        .dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

data_avito = AvitoDataCleanerRealtyFlat().transform( data_avito )

print(len(data_avito))

22288


In [8]:
assert len(data_avito)>0

In [9]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22288 entries, 0 to 22287
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       22288 non-null  int64         
 1   title          22288 non-null  object        
 2   price          22288 non-null  int64         
 3   adr            22288 non-null  object        
 4   description    22288 non-null  object        
 5   obj_name       22288 non-null  object        
 6   ts             22288 non-null  datetime64[ns]
 7   avito_page     22288 non-null  int64         
 8   place          22288 non-null  object        
 9   nrooms         22288 non-null  int64         
 10  floor          22288 non-null  int64         
 11  nfloors        22288 non-null  int64         
 12  area           22288 non-null  float64       
 13  is_studio      22288 non-null  bool          
 14  is_apartment   22288 non-null  bool          
 15  is_part        2228

In [10]:
data_avito.isnull().any()

avito_id         False
title            False
price            False
adr              False
description      False
obj_name         False
ts               False
avito_page       False
place            False
nrooms           False
floor            False
nfloors          False
area             False
is_studio        False
is_apartment     False
is_part          False
is_auction       False
is_openspace     False
is_roof          False
is_SNT           False
priceM           False
is_last_floor    False
dtype: bool

In [11]:
# data_avito.query('nrooms==0')
# data_avito[ data_avito['title'].str.len()<1]

## собираем данные циан.ру

In [12]:
import re
from os import listdir

raw_data_files = sorted([
        cian_raw_data_path+'/'+f 
        for f in listdir(cian_raw_data_path) 
        if re.match(r'.+\.pkl$',f)
        #if re.match(r'.+\.xlsx$',f)
    
    ])
raw_data_files

['data/raw/flat/cian/cian_2022-09-08_14-25_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-12_13-59_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-13_13-59_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-15_10-30_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-16_15-11_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-19_12-40_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-22_09-53_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-26_11-23_raw.pkl',
 'data/raw/flat/cian/cian_2022-10-08_20-39_raw.pkl',
 'data/raw/flat/cian/cian_2022-10-14_15-53_raw.pkl',
 'data/raw/flat/cian/cian_2022-11-01_13-57_raw.pkl',
 'data/raw/flat/cian/cian_2022-11-04_17-14_raw.pkl',
 'data/raw/flat/cian/cian_2022-11-07_10-47_raw.pkl']

In [13]:
data_cian = pd.concat([
        pd.read_pickle(f)
         .dropna()
         .drop_duplicates(['LinkArea','Description',])
         .rename(columns={'page':'cian_page'})
         .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

In [14]:
data_cian.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19318 entries, 0 to 19317
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OfferTitle     19318 non-null  object        
 1   OfferSubtitle  19318 non-null  object        
 2   Deadline       19318 non-null  object        
 3   MainPrice      19318 non-null  object        
 4   PriceInfo      19318 non-null  object        
 5   GeoLabel       19318 non-null  object        
 6   TimeLabel      19318 non-null  object        
 7   LinkArea       19318 non-null  object        
 8   Description    19318 non-null  object        
 9   cian_page      19318 non-null  int64         
 10  ts             19318 non-null  datetime64[ns]
 11  place          19318 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 1.8+ MB


In [15]:
data_cian.isnull().any()

OfferTitle       False
OfferSubtitle    False
Deadline         False
MainPrice        False
PriceInfo        False
GeoLabel         False
TimeLabel        False
LinkArea         False
Description      False
cian_page        False
ts               False
place            False
dtype: bool

In [16]:
swap_title = data_cian['OfferSubtitle'].str.match(r'.*\d+ м², \d+/\d+ эт.*') 

data_cian = pd.concat([
        data_cian[ ~swap_title ],
        data_cian[ swap_title ]
            .rename(columns={'OfferTitle':'OfferSubtitle','OfferSubtitle':'OfferTitle'})
    ]).reset_index(drop=True)


In [17]:
# data_cian

In [18]:
from lib.cian import CianDataCleaner

data_cian = CianDataCleaner().transform( data_cian )

print(len(data_cian))

19318


In [19]:
assert len(data_cian)>0

In [20]:
data_cian.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19318 entries, 0 to 19317
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          19318 non-null  object        
 1   obj_name       19318 non-null  object        
 2   adr            19318 non-null  object        
 3   nrooms         19318 non-null  int64         
 4   floor          19318 non-null  int64         
 5   nfloors        19318 non-null  int64         
 6   area           19318 non-null  float64       
 7   is_studio      19318 non-null  bool          
 8   is_apartment   19318 non-null  bool          
 9   is_part        19318 non-null  bool          
 10  is_auction     19318 non-null  bool          
 11  is_openspace   19318 non-null  bool          
 12  is_roof        19318 non-null  bool          
 13  is_SNT         19318 non-null  bool          
 14  price          19318 non-null  int64         
 15  priceM         1931

In [21]:
data_cian.isnull().any()

title            False
obj_name         False
adr              False
nrooms           False
floor            False
nfloors          False
area             False
is_studio        False
is_apartment     False
is_part          False
is_auction       False
is_openspace     False
is_roof          False
is_SNT           False
price            False
priceM           False
is_last_floor    False
cian_url         False
description      False
cian_page        False
ts               False
place            False
dtype: bool

In [22]:
# data_cian
# data_cian.query('nrooms==0')
# data_cian[ data_cian['title'].str.len()<1 ]
# data_cian[ data_cian['obj_name'].str.len()>0 ][['title','obj_name',]] # .to_csv('tmp/cian_title.csv',sep='\t')|

----

In [23]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
data_cian = data_cian.rename(columns={'cian_url':'url'})

In [24]:
data = pd.concat([data_avito,data_cian]).reset_index(drop=True) 
print(len(data))
assert len(data)>0

41606


In [25]:
data.sample()

,title,price,adr,description,obj_name,ts,avito_page,place,nrooms,floor,...,is_apartment,is_part,is_auction,is_openspace,is_roof,is_SNT,priceM,is_last_floor,url,cian_page
31971,"3-комн. кв., 72 м², 1/9 этаж",9500000,"Севастополь, район Гагаринский, микрорайон Летчики, улица Колобова, 15","Трехкомнатная квартира 72 м2, на 1/9 этаже дома. Расположена в г. Севастополь, Гагаринский район, ул. Колобова, д. 15.\nЭто предложение отлично подойдет людям, ценящим комфорт и близость всей инфраструктуры.\nПозвоните нам, чтобы узнать все детали! \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Гагаринский район, ул. Колобова, д. 15\n-Год постройки дома: 1985\n-Этаж/этажность: 1/9\n-Количество комнат: 3\n-Комнаты: изолированные \n-Общая площадь: 72 м2\n-Жилая площадь: 38,4 м2\n-Комната: 17,8; 11,8 и 8,8 м2\n-Площадь кухни: 7,5 м2\n-Высота потолка: 2,5 м \n-Санузел: раздельный\n-Водоснабжение: центральное \n-Горячее водоснабжение: АГВ\n-Отопление: АГВ\nДополнительная информация: \nТрехкомнатная квартира улучшенной планировки на 1 высоком этаже 9-ти этажного дома, окна выходят во двор.\nОбщая площадь квартиры с учетом лоджий - 72 м.кв. Комнаты - 17,8 м.кв., 11.8 м.кв. с выходом на большую лоджию - 7,5 м.кв. и комната - 8.8 м.кв. Кухня - 7,5 м.кв. с выходом на лоджию - 5,1 м.кв, коридор - 5,1 м.кв, 3,1 м.кв. и 2,6 м.кв, санузел раздельный - 2,6 м.кв. и 1,0 м.кв.\nВ квартире установлено автономное газовое отопление. Квартира в хорошем жилом состоянии.\nВо дворе для детей и взрослых предусмотрены игровые и спортивные площадки. \nПозвоните нам, чтобы назначить просмотр! \nИнфраструктура: \nКвартира расположена в районе с очень развитой инфраструктурой. В шаговой доступности школы 37 и 58, детские сады 111 и 125, детские развивающие центры, салоны красоты, продуктовые супермаркеты, рынок, остановки общественного транспорта, кафе и рестораны. \nУдобная транспортная развязка позволит вам очень быстро добраться в любой район города: центр - 15 минут, пляж ""Омега и парк ""Победы"" - 10 минут.\nПодходит для наличного расчета, под гражданскую ипотеку, материнский капитал, жилищные сертификаты. \nДокументы РФ проверены юристами и готовы к сделке. \nПрофессиональное сопровождение до получения права собственности.\nДобавьте предложение в закладки, чтобы не потерять!",,2022-09-22 09:53:39.964462,NaN,sevastopol,3,1,...,False,False,False,False,False,False,9.50,False,https://sevastopol.cian.ru/sale/flat/273182796/,41.00


## обновляем таблицу адресов

In [26]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [27]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [28]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/41606 [00:00<?, ?it/s]

  0%|          | 0/41606 [00:00<?, ?it/s]

In [29]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
23830,"Севастополь, район Гагаринский, микрорайон Летчики, улица Астана Кесаева, 3","Севастополь, район Гагаринский, Летчики, улица Астана Кесаева, 3"
10292,"СНТ Бриз, 137","Севастополь, СНТ Бриз, 137"
40803,"Севастополь, район Нахимовский, микрорайон Корабельная сторона, улица Розы Люксембург, 84","Севастополь, район Нахимовский, Корабельная сторона, улица Розы Люксембург, 84"
16796,"шоссе Лабораторное, д. 33, секц. 4","Севастополь, шоссе Лабораторное, д. 33, секц. 4"
12633,"улица Генерала Крейзера, д. 8, корп. 5","Севастополь, улица Генерала Крейзера, д. 8, корп. 5"
8617,"улица Сенявина, 5","Севастополь, улица Сенявина, 5"
4445,"улица Бориса Михайлова, 3","Севастополь, улица Бориса Михайлова, 3"
13550,"улица Горпищенко, 109","Севастополь, улица Горпищенко, 109"
38463,"Севастополь, район Ленинский, микрорайон Острякова, улица Хрусталева, 11","Севастополь, район Ленинский, Острякова, улица Хрусталева, 11"
24348,"Севастополь, район Гагаринский, микрорайон Камышовая бухта, улица Героев Бреста, 37","Севастополь, район Гагаринский, Камышовая бухта, улица Героев Бреста, 37"


In [30]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2022-11-07 12:11:50 | LocationUpdater: 7073 addresses in index
[INFO    ] 2022-11-07 12:11:50 | LocationUpdater: 7096 addresses total
[INFO    ] 2022-11-07 12:11:50 | LocationUpdater: 7073 addresses defined
[INFO    ] 2022-11-07 12:11:50 | LocationUpdater: 23 addresses undefined


  0%|          | 0/23 [00:00<?, ?it/s]

[INFO    ] 2022-11-07 12:11:56 | LocationUpdater: 23 new addresses found


In [31]:
print( len(loc) )
loc.sample(3)

7096


,latitude,longitude,adr
6769,44.580925,33.49391,"Севастополь,ул. Кедрина, 12"
1169,44.58,33.46,"Севастополь, улица Колобова, 18"
6990,44.592987,33.430615,"Севастополь, район Гагаринский, Омега, улица Челнокова, 17/4"


## дополняем данные геометкой

In [32]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

In [33]:
data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

всего записей: 41606
записей без геометки: 0


In [34]:
assert len(data)>0
data.to_pickle(result_file_path)

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41606 entries, 0 to 41605
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          41606 non-null  object        
 1   price          41606 non-null  int64         
 2   adr_orig       41606 non-null  object        
 3   description    41606 non-null  object        
 4   obj_name       41606 non-null  object        
 5   ts             41606 non-null  datetime64[ns]
 6   avito_page     22288 non-null  float64       
 7   place          41606 non-null  object        
 8   nrooms         41606 non-null  int64         
 9   floor          41606 non-null  int64         
 10  nfloors        41606 non-null  int64         
 11  area           41606 non-null  float64       
 12  is_studio      41606 non-null  bool          
 13  is_apartment   41606 non-null  bool          
 14  is_part        41606 non-null  bool          
 15  is_auction     4160